<a href="https://colab.research.google.com/github/shradaa/PyTrainingDWS/blob/master/Text_Classification_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/Kaggledata.csv')
df.head()

,review,sentiment
0,Shanghai is also really exciting (precisely -...,positive
1,"Recession hit Veronique Branquinho, she has to...",negative
2,happy bday!,positive
3,http://twitpic.com/4w75p - I like it!!,positive
4,that`s great!! weee!! visitors!,positive


In [2]:
pip install torch

In [3]:
pip install transformers

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def process_data(row):
        # Clean the text
        text = row['review']
        text = str(text)
        text = ' '.join(text.split())
        # Get tokens
        encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
        # Convert string to integers
        label = 0
        if row['sentiment'] == 'positive':
            label += 1

        encodings['label'] = label
        encodings['text'] = text

        return encodings

In [6]:
 print(process_data({'review': 'happy bday!','sentiment': 'positive'}))

{'input_ids': [101, 3407, 1038, 10259, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
# Store the encodings into an array to generate dataset
processed_data = []

for i in range(len(df[:1000])):
  processed_data.append(process_data(df.iloc[i]))

In [8]:
from sklearn.model_selection import train_test_split
new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
        new_df,
        test_size=0.2,
        random_state=2022
    )

In [9]:
pip install datasets

In [10]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [11]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
pip install transformers[torch]

In [13]:
pip install accelerate -U

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_hg,
        eval_dataset=valid_hg,
        tokenizer=tokenizer
    )

In [15]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.290602


Epoch,Training Loss,Validation Loss
1,No log,0.290602
2,No log,0.320688
3,No log,0.370772


TrainOutput(global_step=300, training_loss=0.2338616943359375, metrics={'train_runtime': 3463.6353, 'train_samples_per_second': 0.693, 'train_steps_per_second': 0.087, 'total_flos': 157866633216000.0, 'train_loss': 0.2338616943359375, 'epoch': 3.0})

In [16]:
trainer.evaluate()

{'eval_loss': 0.3707715570926666,
 'eval_runtime': 107.6131,
 'eval_samples_per_second': 1.859,
 'eval_steps_per_second': 0.232,
 'epoch': 3.0}

In [17]:
model.save_pretrained('./model/')

In [18]:
from transformers import AutoModelForSequenceClassification

new_model = AutoModelForSequenceClassification.from_pretrained('./model/')

In [19]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [20]:
import torch
import numpy as np

def get_prediction(text):
        encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

        outputs = new_model(**encoding)

        logits = outputs.logits

        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(logits.squeeze().cpu())
        probs = probs.detach().numpy()
        label = np.argmax(probs, axis=-1)

        if label == 1:
            return {
                'sentiment': 'Positive',
                'probability': probs[1]
            }
        else:
            return {
                'sentiment': 'Negative',
                'probability': probs[0]
            }

In [21]:
get_prediction('This makes me so sad')

{'sentiment': 'Negative', 'probability': 0.9767363}

In [24]:
get_prediction('I donot know what to do')

{'sentiment': 'Negative', 'probability': 0.9690111}